This script generates an ensemble of frequency-current curves. The predictions are made using $\texttt{NOBLE}$, sampling the neuron models, $\{\text{HoF}^{train}\}$, experienced during training, and across stimulus amplitudes ranging from depolarized to hyperpolarized dynamics.

In [1]:
import numpy as np
import torch

from utils.input_builder import extract_scaled_e_features
from utils.frequency_current_curve import generate_frequency_current_curve
from utils.plotting import plot_fi_curve  
from utils.model_utils import load_model

**Step 1: Set up parameters**

In [ ]:
device = "cuda"

features_to_embed = ["slope", "intercept"]

embedding_config = {"sine_embeddings_freq": 9,
                    "scale_sine_embeddings": "freq",
                    "amplitude_embeddings": False,
                    "hof_model_embeddings": 1,
                    "e_features_to_embed": features_to_embed}

**Step 2: Load electrophysiological features**

In [ ]:
normalised_features = extract_scaled_e_features(neuron_identifier="PVALB_689331391", 
                                                path_to_features='../data/e_features/pvalb_689331391_ephys_sim_features.csv', 
                                                features_to_embed=features_to_embed)

**Step 3: Load trained $\texttt{NOBLE}$ model**

In [ ]:
model_path = 'noble_models/FNO_nmodes-256_in-23_out-1_nlayers-12_projectionratio-4_hc-24_AmpEmbeddings-FreqScaledNeRF-nfreq-9_HoFEmbeddings-FreqScaledNeRF-nfreq-1_bestepoch-296.pth'
model = load_model(model_path, device)

Loading FNO model with modes=256, in_channels=23, out_channels=1, nlayers=12, projection_ratio=4,  hidden_channels=24, device=cpu.



**Step 4: Define stimulus amplitudes for construcing the frequency-current curve**

In [ ]:
amplitudes = np.array([-1.1, -0.7, -0.5, -0.3, -0.1, -0.05])
amplitudes = np.concatenate((amplitudes, np.linspace(0.001, 0.5, 500)))
amplitudes = np.concatenate((amplitudes, np.linspace(0.5, 2.7, 50)))

**Step 5: Collect embeddings from $\{\text{HoF}^{train}\}$ models** 

In [9]:
known_hof_models     = np.arange(0, 60)
hof_test             = np.array([32, 2, 12, 40, 5, 52, 21, 29, 16, 37])
hof_train            = np.setdiff1d(known_hof_models, hof_test)
features_train       = normalised_features[normalised_features["hof_model"].isin(hof_train)]

sampled_models = {"slope": torch.tensor(features_train["slope"].values, dtype=torch.float32), 
                    "intercept": torch.tensor(features_train["intercept"].values, dtype=torch.float32)}

**Step 6: Generate frequency-current curves**

In [10]:
fi_curves = generate_frequency_current_curve(amplitudes=amplitudes, 
                                             sampled_models=sampled_models, 
                                             noble_model=model, 
                                             normalised_features=normalised_features, 
                                             features_to_embed=features_to_embed, 
                                             embedding_config=embedding_config, 
                                             device=device)

plot_fi_curve(fi_curves=fi_curves, 
              amplitudes=amplitudes)

processed amplitudes 0%


KeyboardInterrupt: 